In [1]:
import re
import requests
import pandas as pd

# URLを指定
url = "https://nenji-toukei.com/n/kiji/10069"

# ウェブページのHTMLを取得
response = requests.get(url)
html_content = response.text

# JavaScriptの配列内のデータを正規表現で抽出
data_pattern = r"google.visualization.arrayToDataTable\((.*?)\);"
data_match = re.search(data_pattern, html_content, re.DOTALL)

if data_match:
    data_string = data_match.group(1).strip()
    
    # 数字のみを抽出
    numbers = re.findall(r"\b\d+\b", data_string)
    
# 1960年から2010年までのデータのみを抽出
filtered_numbers = [(int(numbers[i]), int(numbers[i+2])) for i in range(0, len(numbers), 3) if 1960 <= int(numbers[i]) <= 2010 and int(numbers[i]) % 10 == 0]
    
# DataFrameに変換
df = pd.DataFrame(filtered_numbers, columns=["Year", "Inflation-adjusted price(yen)"])

# "Inflation-adjusted price"列の値を2倍する（タバコ10本入りの値だったため現在の20本あたりの値段にする）
df["Inflation-adjusted price(yen)"] = df["Inflation-adjusted price(yen)"] * 2

# 新しい行のデータを作成(2020年のデータのみ別サイトから引用)
new_row = {"Year": 2020, "Inflation-adjusted price(yen)": 560}

# DataFrameに新しい行を追加
df.loc[len(df)] = new_row

# 結果を表示
print(df)

   Year  Inflation-adjusted price(yen)
0  1960                            742
1  1970                            366
2  1980                            226
3  1990                            224
4  2000                            238
5  2010                            336
6  2020                            560


In [2]:
import matplotlib.pyplot as plt

# DataFrameから表を描画
plt.figure(figsize=(6, 6))
plt.table(cellText=df.values,
          colLabels=df.columns,
          cellLoc = 'center', 
          loc='center')
plt.axis('off')  # 軸を非表示にする

# 表を画像ファイルとして保存
plt.savefig('Japanese_tobbaco_price.png', bbox_inches='tight', pad_inches=0)

# グラフを表示（不要なので非表示にする）
plt.close()